In [1]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e3a414254128cea0dc0b91efdad9124c6468f29d1a1960959b0d576ab62987b2
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import os
import sys
import pyspark

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
#grant permission (for gdrive use only)
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
cd gdrive/MyDrive/

/content/gdrive/MyDrive


In [5]:
#load housing dataset from csv file
from pyspark.sql import SparkSession

ss = SparkSession.builder.appName('housing_data').getOrCreate()
df_without_header = ss.read.option('inferSchema', True).option('header', False).csv('housing_data.csv')
df_without_header = df_without_header.na.drop()
df_without_header.show()

+--------------------+------+-------------------+--------+---+---+---+--------------------+----------------+--------------------+------------------+------------------+--------------------+--------------+----+----+
|                 _c0|   _c1|                _c2|     _c3|_c4|_c5|_c6|                 _c7|             _c8|                 _c9|              _c10|              _c11|                _c12|          _c13|_c14|_c15|
+--------------------+------+-------------------+--------+---+---+---+--------------------+----------------+--------------------+------------------+------------------+--------------------+--------------+----+----+
|{E104A9E7-1D6A-4D...| 36500|1995-06-09 00:00:00| FY4 1DL|  F|  N|  L|      CLARENCE COURT|              28|    RAWCLIFFE STREET|         BLACKPOOL|         BLACKPOOL|           BLACKPOOL|     BLACKPOOL|   A|   A|
|{748F870E-C337-40...| 74000|1995-01-12 00:00:00| RH2 9NF|  F|  Y|  L|SOMERS PLACE, 83 ...|         FLAT 21|        REIGATE HILL|           REIG

In [6]:
#add column names from kaggle dataset page
col_names=['Transaction_unique_identifier', 'price', 'Date_of_Transfer', 'postcode', 'Property_Type', 'Old/New',
'Duration', 'PAON', 'SAON', 'Street', 'Locality', 'Town/City', 'District', 'County', 'PPDCategory_Type',
'Record_Status - monthly_file_only']
housing_df = df_without_header.toDF(*col_names)

# use first 500000 entries due to limitations in compute power
housing_df = ss.createDataFrame(housing_df.head(200000), housing_df.schema)

housing_df.show()
print(housing_df.count())

+-----------------------------+------+-------------------+--------+-------------+-------+--------+--------------------+----------------+--------------------+------------------+------------------+--------------------+--------------+----------------+---------------------------------+
|Transaction_unique_identifier| price|   Date_of_Transfer|postcode|Property_Type|Old/New|Duration|                PAON|            SAON|              Street|          Locality|         Town/City|            District|        County|PPDCategory_Type|Record_Status - monthly_file_only|
+-----------------------------+------+-------------------+--------+-------------+-------+--------+--------------------+----------------+--------------------+------------------+------------------+--------------------+--------------+----------------+---------------------------------+
|         {E104A9E7-1D6A-4D...| 36500|1995-06-09 00:00:00| FY4 1DL|            F|      N|       L|      CLARENCE COURT|              28|    RAWCLIFFE S

In [7]:
#data schema
housing_df.printSchema()

root
 |-- Transaction_unique_identifier: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- Date_of_Transfer: timestamp (nullable = true)
 |-- postcode: string (nullable = true)
 |-- Property_Type: string (nullable = true)
 |-- Old/New: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- PAON: string (nullable = true)
 |-- SAON: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Town/City: string (nullable = true)
 |-- District: string (nullable = true)
 |-- County: string (nullable = true)
 |-- PPDCategory_Type: string (nullable = true)
 |-- Record_Status - monthly_file_only: string (nullable = true)



In [8]:
#get range statistics related to data purely for report writing
import pandas as pd
from pyspark.sql import functions as F


pandas_df = housing_df.toPandas()
min_values = pandas_df.min()
max_values = pandas_df.max()

unique = []
cols = []
for col in pandas_df.columns:
  unique.append(len(pandas_df[col].unique()))
  cols.append(col)

df_stats = pd.DataFrame({'Name' : cols, 'Min' : min_values, 'Max' : max_values, 'Unique values' : unique})
df_stats

,Name,Min,Max,Unique values
Transaction_unique_identifier,Transaction_unique_identifier,{00007F1A-EDE3-4EA1-982C-064180AFAC26},{FFFF5180-5F89-4DDC-906F-E6579B00B9C9},200000
price,price,500,4500000,6525
Date_of_Transfer,Date_of_Transfer,1995-01-01 00:00:00,1997-12-31 00:00:00,1079
postcode,postcode,AL1 1BH,YO8 9QR,58025
Property_Type,Property_Type,D,T,5
Old/New,Old/New,N,Y,2
Duration,Duration,F,U,3
PAON,PAON,"(MILTON), 38","ZULEIKA HOUSE, 235",31735
SAON,SAON,(ANDREWS),YORK COTTAGE 2,4676
Street,Street,AALTEN AVENUE,ZULLA ROAD,21946


In [9]:
#get size stats and category statistics to decide which attributes are useful in computation
from pyspark.sql.functions import col

#total entries
print("Housing data contains " + str(housing_df.count()) + " entries")

#Date_of_Transfer
transfer_count =  housing_df.select('Date_of_Transfer').distinct().count()
print("There are " + str(transfer_count) + " entries based on unique Date_of_Transfer")


Housing data contains 200000 entries
There are 1079 entries based on unique Date_of_Transfer


In [10]:
#find the number of unique features of each value
dt_cols = []

def count_cols(df_col):
    col_count = housing_df.select(df_col).distinct().count()
    print("There are " + str(col_count) + " entries based on unique " + str(df_col) + " values")
    if col_count < 2000:
      dt_cols.append(df_col)

for i in col_names:
    count_cols(i)


There are 200000 entries based on unique Transaction_unique_identifier values
There are 6525 entries based on unique price values
There are 1079 entries based on unique Date_of_Transfer values
There are 58025 entries based on unique postcode values
There are 5 entries based on unique Property_Type values
There are 2 entries based on unique Old/New values
There are 3 entries based on unique Duration values
There are 31735 entries based on unique PAON values
There are 4676 entries based on unique SAON values
There are 21946 entries based on unique Street values
There are 6177 entries based on unique Locality values
There are 1065 entries based on unique Town/City values
There are 453 entries based on unique District values
There are 128 entries based on unique County values
There are 2 entries based on unique PPDCategory_Type values
There are 1 entries based on unique Record_Status - monthly_file_only values


In [11]:
#drop Transaction_unique_identifier due to there not being enough non-unique column entries
housing_df = housing_df.drop('Transaction_unique_identifier')
housing_df = housing_df.drop('Record_Status - monthly_file_only')
#housing_df = housing_df.drop('PPDCategory_Type values')

In [12]:
#get list of string entries from schema
string_type_columns = []
for col in housing_df.dtypes:
    #print(col[0]+" , "+col[1])
    if(col[1] == 'string'):
        string_type_columns.append(col[0])
print(string_type_columns)

['postcode', 'Property_Type', 'Old/New', 'Duration', 'PAON', 'SAON', 'Street', 'Locality', 'Town/City', 'District', 'County', 'PPDCategory_Type']


In [13]:
#encode all columns with string datatype
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

"""def encode_values(column, df_initial):
    indexer = StringIndexer(inputCol= column, outputCol= column + "encoded")
    df_encoded = indexer.fit(df_initial).transform(df_initial)
    df_encoded = df_encoded.drop(column).withColumnRenamed(column + "encoded", column)
    df_encoded.show()
    return df_encoded """

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(housing_df) for column in string_type_columns]
pipeline = Pipeline(stages=indexers)
df_encoded = pipeline.fit(housing_df).transform(housing_df)
df_encoded = df_encoded.drop(*string_type_columns)
df_encoded.show()

"""
encode_values('postcode')
encode_values('Property_Type')
#encode_values('Old/New')
housing_df.show()
"""

+------+-------------------+--------------+-------------------+-------------+--------------+----------+----------+------------+--------------+---------------+--------------+------------+----------------------+
| price|   Date_of_Transfer|postcode_index|Property_Type_index|Old/New_index|Duration_index|PAON_index|SAON_index|Street_index|Locality_index|Town/City_index|District_index|County_index|PPDCategory_Type_index|
+------+-------------------+--------------+-------------------+-------------+--------------+----------+----------+------------+--------------+---------------+--------------+------------+----------------------+
| 36500|1995-06-09 00:00:00|        8099.0|                0.0|          0.0|           0.0|     562.0|      92.0|      4503.0|          71.0|           98.0|         137.0|        58.0|                   0.0|
| 74000|1995-01-12 00:00:00|         722.0|                0.0|          1.0|           0.0|    1390.0|      33.0|      1403.0|          96.0|          123.0|  

"\nencode_values('postcode')\nencode_values('Property_Type')\n#encode_values('Old/New')\nhousing_df.show()\n"

In [14]:
#encode all columns with timestamp datatype
timestamp_type_columns = []
for col in housing_df.dtypes:
    #print(col[0]+" , "+col[1])
    if(col[1] == 'timestamp'):
        timestamp_type_columns.append(col[0])
print(timestamp_type_columns)

['Date_of_Transfer']


In [15]:
from pyspark.sql.functions import col, unix_timestamp
from pyspark.sql.functions import udf
from pyspark.sql.functions import *

#df_encoded.withColumn('Date_of_Transfer_encoded', unix_timestamp(col('Date_of_Transfer'), format='yyyy-MM-dd'))
#df_encoded = df_encoded.drop('Date_of_Transfer').withColumnRenamed('Date_of_Transfer' + "encoded", 'Date_of_Transfer')

extract_year = udf(lambda x: x[0:4])

df_encoded = df_encoded.withColumn('Date_of_Transfer', to_timestamp('Date_of_Transfer').cast('string'))
df_encoded = df_encoded.withColumn('Date_of_Transfer', extract_year('Date_of_Transfer').cast("integer"))
df_encoded.show(1)


+-----+----------------+--------------+-------------------+-------------+--------------+----------+----------+------------+--------------+---------------+--------------+------------+----------------------+
|price|Date_of_Transfer|postcode_index|Property_Type_index|Old/New_index|Duration_index|PAON_index|SAON_index|Street_index|Locality_index|Town/City_index|District_index|County_index|PPDCategory_Type_index|
+-----+----------------+--------------+-------------------+-------------+--------------+----------+----------+------------+--------------+---------------+--------------+------------+----------------------+
|36500|            1995|        8099.0|                0.0|          0.0|           0.0|     562.0|      92.0|      4503.0|          71.0|           98.0|         137.0|        58.0|                   0.0|
+-----+----------------+--------------+-------------------+-------------+--------------+----------+----------+------------+--------------+---------------+--------------+-------

In [16]:
df_encoded.write.csv("housing_modified.csv", mode='overwrite')

In [17]:
#find correlation coefficient between all variables to identify variables to keep for linear regression
from pyspark.sql.functions import *

df_linreg = df_encoded.alias('df_linreg')
all_columns = []
for col in df_encoded.dtypes:
        all_columns.append(col[0])
#print(all_columns)
for col in all_columns:
    val = df_encoded.stat.corr(col, 'price')
    print("The correlation between price and " + col + " is:", val)
    #categortical variables don't show high correlation, as expected, therefore they cannot be used for linear regression
    if val < 0.1 and val > -0.1:
        df_linreg = df_linreg.drop(col)

print("\n Dataframe for linear regression: ")
df_linreg.show(1)


The correlation between price and price is: 1.0
The correlation between price and Date_of_Transfer is: 0.07736112392334328
The correlation between price and postcode_index is: -0.039206651590024545
The correlation between price and Property_Type_index is: 0.06068293401736996
The correlation between price and Old/New_index is: 0.08338657061086012
The correlation between price and Duration_index is: 0.02861651482256954
The correlation between price and PAON_index is: -0.06314254054536322
The correlation between price and SAON_index is: 0.07712298571838136
The correlation between price and Street_index is: -0.07103708032651991
The correlation between price and Locality_index is: -0.07625870363489233
The correlation between price and Town/City_index is: -0.17226351079019528
The correlation between price and District_index is: -0.24599619576467302
The correlation between price and County_index is: -0.2263188139085693
The correlation between price and PPDCategory_Type_index is: -0.0021987358

In [18]:
#normalize all columns in the dataframe
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType


unlist = udf(lambda x: float(list(x)[0]), DoubleType())

for col in df_linreg.dtypes:
  i = col[0]
  #print(i)
  assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")
  scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")
  pipeline = Pipeline(stages=[assembler, scaler])
  df_linreg = pipeline.fit(df_linreg).transform(df_linreg).withColumn(i + '_Scaled2', unlist(i+"_Scaled")).drop(i+"_Vect", i + "_Scaled", i)

df_linreg.show(5)




+--------------------+-----------------------+----------------------+--------------------+
|       price_Scaled2|Town/City_index_Scaled2|District_index_Scaled2|County_index_Scaled2|
+--------------------+-----------------------+----------------------+--------------------+
|0.008000888987665296|    0.09210526315789473|   0.30309734513274333| 0.45669291338582674|
|0.016335148349816645|    0.11560150375939848|   0.07743362831858407|0.015748031496062992|
| 0.00277808645405045|    0.08740601503759399|   0.11504424778761062|0.023622047244094488|
|0.006545171685742861|   9.398496240601503E-4|  0.008849557522123894| 0.05511811023622047|
|0.005778419824424936|    0.02537593984962406|    0.3163716814159292| 0.22834645669291337|
+--------------------+-----------------------+----------------------+--------------------+
only showing top 5 rows



In [19]:
#apply linear regression to the filtered columns
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

#extract features as vector and create the model
assembler = VectorAssembler(inputCols=['Town/City_index_Scaled2', 'District_index_Scaled2', 'County_index_Scaled2'], outputCol="features")
reg_data = assembler.transform(df_linreg)
final_data = reg_data.select("features", "price_Scaled2")
final_data.show(5)
train_data, test_data = final_data.randomSplit([0.99, 0.01], 24)

train_data.show(5)
linear_reg = LinearRegression(featuresCol="features", labelCol="price_Scaled2", predictionCol="predicted_price")
lr_model = linear_reg.fit(train_data)

#training accuracy
train_pred = lr_model.transform(train_data)
#train_pred.show(5)
evaluator = RegressionEvaluator(labelCol="price_Scaled2", predictionCol="predicted_price", metricName="rmse")
rmse = evaluator.evaluate(train_pred)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(rmse))
evaluator_r2 = RegressionEvaluator(labelCol="price_Scaled2", predictionCol="predicted_price", metricName="r2")
r2 = evaluator_r2.evaluate(train_pred)
print("R-squared (R2) on train data: {:.3f}".format(r2))

#testing accuracy
test_pred = lr_model.transform(test_data)
#test_pred.show(5)
evaluator = RegressionEvaluator(labelCol="price_Scaled2", predictionCol="predicted_price", metricName="rmse")
rmse = evaluator.evaluate(test_pred)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(rmse))
evaluator_r2 = RegressionEvaluator(labelCol="price_Scaled2", predictionCol="predicted_price", metricName="r2")
r2 = evaluator_r2.evaluate(test_pred)
print("R-squared (R2) on test data: {:.3f}".format(r2))

+--------------------+--------------------+
|            features|       price_Scaled2|
+--------------------+--------------------+
|[0.09210526315789...|0.008000888987665296|
|[0.11560150375939...|0.016335148349816645|
|[0.08740601503759...| 0.00277808645405045|
|[9.39849624060150...|0.006545171685742861|
|[0.02537593984962...|0.005778419824424936|
+--------------------+--------------------+
only showing top 5 rows

+---------+--------------------+
| features|       price_Scaled2|
+---------+--------------------+
|(3,[],[])|0.007667518613179...|
|(3,[],[])|0.007667518613179...|
|(3,[],[])|0.007889765529503278|
|(3,[],[])| 0.00833425936215135|
|(3,[],[])|0.008445382820313369|
+---------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data: 0.018
R-squared (R2) on train data: 0.068
Root Mean Squared Error (RMSE) on test data: 0.018
R-squared (R2) on test data: 0.051


In [ ]:
#filtering columns to use in decision trees based on unique values
dt_cols = []
col_names = [col[0] for col in df_encoded.dtypes]

def count_cols(df_col, price_count):
    col_count = df_encoded.select(df_col).distinct().count()
    print("There are " + str(col_count) + " entries based on unique " + str(df_col) + " values")
    if col_count < price_count:
      dt_cols.append(df_col)

price_count = df_encoded.select('price').distinct().count()
for i in col_names:
    count_cols(i, price_count)

print(dt_cols)

There are 23532 entries based on unique price values
There are 116948 entries based on unique postcode_index values
There are 5 entries based on unique Property_Type_index values
There are 2 entries based on unique Old/New_index values
There are 3 entries based on unique Duration_index values
There are 62851 entries based on unique PAON_index values
There are 14575 entries based on unique SAON_index values
There are 37308 entries based on unique Street_index values
There are 9000 entries based on unique Locality_index values
There are 1109 entries based on unique Town/City_index values
There are 456 entries based on unique District_index values
There are 130 entries based on unique County_index values
There are 2 entries based on unique PPDCategory_Type_index values
['Property_Type_index', 'Old/New_index', 'Duration_index', 'SAON_index', 'Locality_index', 'Town/City_index', 'District_index', 'County_index', 'PPDCategory_Type_index']


In [ ]:

#create vector assembler
df_decisiontrees = df_encoded.alias('df_encoded')
assembler = VectorAssembler(inputCols = dt_cols, outputCol="features")
df_dt = assembler.transform(df_decisiontrees)
df_dt.show(10)

#split into train/test
train_data, test_data = dt_dt.randomSplit([0.75, 0.25], 15)
train_data.show(5)

#create decision tree classifier
data = df_dt.select("features", "price")
dt = DecisionTreeRegressor(maxBins = price_count).setLabelCol("price").setFeaturesCol("features")
dt_regressor = dt.fit(train_data)

#transform training data
train_pred = dt_regressor.transform(train_data)
train_pred.show(5)

#training set accuracy
dtevaluator = RegressionEvaluator(predictionCol="prediction", labelCol="price", metricName="rmse")
rmse = dtevaluator.evaluate(train_pred)
print("Root Mean Squared Error (RMSE) on train data: {:.3f}".format(rmse))
evaluator_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(train_pred)
print("R-squared (R2) on train data: {:.3f}".format(r2))



#transform testing data
test_pred = dt_regressor.transform(test_data)
test_pred.show(5)


#test set accuracy
dtevaluator = RegressionEvaluator(predictionCol="prediction", labelCol="price", metricName="rmse")
rmse = dtevaluator.evaluate(test_pred)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(rmse))
evaluator_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(test_pred)
print("R-squared (R2) on test data: {:.3f}".format(r2))


+------+--------------+-------------------+-------------+--------------+----------+----------+------------+--------------+---------------+--------------+------------+----------------------+--------------------+
| price|postcode_index|Property_Type_index|Old/New_index|Duration_index|PAON_index|SAON_index|Street_index|Locality_index|Town/City_index|District_index|County_index|PPDCategory_Type_index|            features|
+------+--------------+-------------------+-------------+--------------+----------+----------+------------+--------------+---------------+--------------+------------+----------------------+--------------------+
| 36500|       17045.0|                0.0|          0.0|           0.0|     350.0|      91.0|      7406.0|          67.0|           91.0|         126.0|        60.0|                   0.0|[0.0,0.0,0.0,91.0...|
| 74000|        3847.0|                0.0|          1.0|           0.0|    4189.0|      33.0|      2403.0|         139.0|          172.0|          55.0|   